In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

Mounted at /content/gdrive/


In [2]:
%cd '/content/gdrive/My Drive/IDLSProject-main'

/content/gdrive/My Drive/IDLSProject-main


In [3]:
import pandas as pd
import os
import pickle

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_amazon.pkl'), "rb") as f:
    word_dict = pickle.load(f)

In [4]:
import pandas as pd
import os
train = pd.read_csv(os.path.join(data_dir, 'train_amazon.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_amazon.csv'), header=None, names=None)
print(train.shape, test_sample.shape)

(30000, 502) (20000, 502)


In [5]:
from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

((30000, 502), (10000, 502), (10000, 502))

In [6]:
import torch
import torch.utils.data

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float().squeeze()
train_X = torch.from_numpy(train.drop([0], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float().squeeze()
val_X = torch.from_numpy(val.drop([0], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float().squeeze()
test_X = torch.from_numpy(test.drop([0], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10000])


In [7]:
import torch.nn as nn
import numpy as np

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, trial,vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        hidden_dim = trial.suggest_int("hidden_dim",16,256)
        embedding_dim = trial.suggest_int("embedding_dim",16,128)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig(out.squeeze())

In [ ]:
def write_to_csv(trail_num,epochs, train_loss, val_loss, val_acc, time_train):
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv("./lstm-amazon-hyperband-trails/"+str(trail_num)+".csv")
    
# def append_to_csv(epochs, accuracy):
#     acc = [accuracy for i in range(epochs)]
#     df_csv = pd.read_csv(file_name)
#     df_csv['Test_Accuracy']  = accuracy
#     df_metrics.to_csv(filename)

In [10]:
!pip install optuna

     |████████████████████████████████| 308 kB 5.7 MB/s 
     |████████████████████████████████| 210 kB 43.5 MB/s 
     |████████████████████████████████| 81 kB 9.2 MB/s 
     |████████████████████████████████| 78 kB 6.7 MB/s 
     |████████████████████████████████| 146 kB 43.1 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
     |████████████████████████████████| 112 kB 46.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=085e573195c00c0993a5073111f5aa277e41ec89803a799ac75c54e6baa99561
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [11]:
import time
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import torch.optim as optim
import numpy as np
epochs = 10

# Define an objective function to be minimized.
def objective(trial):
  optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
  #epochs = trial.suggest_int("epochs",5,15)
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
  #momentum = trial.suggest_uniform("momentum", 0.0, 1.0)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = LSTMClassifier(trial,5000).to(device)
  trial.set_user_attr(key="best_model", value=model)
  optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
  loss_fn = torch.nn.BCELoss()
  train_loss_epoch = []
  train_acc_epoch = []
  val_loss_epoch = []
  val_accuracy_epoch = []
  time_train = []
  for epoch in range(epochs):
    model.train()
    start = time.time()
    total_loss = 0
    train_acc = 0
    total = 0
    correct = 0
    for batch in train_dl:         
      batch_X, batch_y = batch
      batch_X = batch_X.to(device)
      batch_y = batch_y.to(device)
      optimizer.zero_grad()
      prediction = model(batch_X)
      loss = loss_fn(prediction, batch_y)
      loss.backward()
      optimizer.step()
      result = np.round(prediction.detach().cpu())
      total_loss += loss.data.item()
      total += batch_y.size(0)
      correct += (result == batch_y.cpu()).sum().item()
      train_acc = correct/total
    train_loss_epoch.append(np.round(total_loss / len(train_dl), 3))
    train_acc_epoch.append(np.round(train_acc*100,3))
    print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_dl)))
    with torch.no_grad():
      model.eval()
      correct = 0
      total = 0
      val_loss = []
      for inputs, labels in val_dl:
        inputs_val, labels_val = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        prediction = model(inputs_val)
        loss = loss_fn(prediction, labels_val)
        val_loss.append(np.round(loss.item(),3))
        result = np.round(prediction.cpu())
        total += labels_val.size(0)
        correct += (result == labels_val.cpu()).sum().item()
      val_accuracy_epoch.append(np.round((correct/total)*100, 3))
      val_loss_epoch.append(np.round(np.mean(val_loss),3))
      end = time.time() - start
      print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(correct/total))
      time_train.append(np.round(end,3))
      val_acc = correct/total
  write_to_csv(epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_accuracy_epoch, time_train)
  return val_acc


In [12]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [ ]:
import optuna
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=10,callbacks=[callback])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
best_model=study.user_attrs["best_model"]
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

[I 2022-05-14 08:26:57,382] A new study created in memory with name: no-name-7ca76f20-d121-4586-85d7-223972657c3a


Epoch: 0, BCELoss: 0.49429892693956695
Val Loss: 0.349 	Val Acc: 0.847


In [ ]:
def test(model, test_dl, epochs):
    model.eval()
    correct = 0
    total = 0
#     results = []
#     labels = []
    with torch.no_grad():
        for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
    print("Accuracy:", correct/total)

In [ ]:
test(best_model, test_dl, epochs)